In [ ]:
# =============================================================================
# CELL A.1: SETUP & CONFIGURATION + PLOT UTILITIES
# =============================================================================
import os
import sys
from pathlib import Path
import json
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_theme(style="whitegrid", palette="viridis")

# --- AUTO-DETECT ENVIRONMENT ---
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    # Colab: Download data if needed
    import requests, zipfile
    DATA_URL = "https://bkuteam.site/data.zip"
    BASE_DIR = Path("/content/ABC2026")
    BASE_DIR.mkdir(parents=True, exist_ok=True)
    DATA_ROOT = BASE_DIR / "ABC2026 Sozolab Challenge"
    DATASET_DIR = DATA_ROOT / "Dataset"
    
    if not (DATASET_DIR.exists() and any(DATASET_DIR.glob("*.csv"))):
        print("📥 Downloading data.zip...")
        ZIP_PATH = BASE_DIR / "data.zip"
        resp = requests.get(DATA_URL, stream=True)
        resp.raise_for_status()
        with open(ZIP_PATH, "wb") as f:
            for chunk in resp.iter_content(chunk_size=8192):
                if chunk: f.write(chunk)
        print("📦 Extracting...")
        with zipfile.ZipFile(ZIP_PATH, "r") as zf:
            zf.extractall(BASE_DIR)
    print(f"✅ [COLAB] Data ready at: {DATASET_DIR}")
else:
    # Local: Use workspace path
    WORKSPACE = Path(r"E:\project\ABC2026")
    DATA_ROOT = WORKSPACE / "ABC2026 Sozolab Challenge"
    DATASET_DIR = DATA_ROOT / "Dataset"
    print(f"✅ [LOCAL] Data at: {DATASET_DIR}")

DATA_PATH = str(DATASET_DIR)
OUTPUT_PATH = str(DATA_ROOT)

# --- CREATE FIGURES DIRECTORY ---
FIGURES_DIR = DATA_ROOT / "figures"
FIGURES_DIR.mkdir(exist_ok=True)
print(f"📊 Figures will be saved to: {FIGURES_DIR}")

# --- PLOT METADATA TRACKER ---
PLOT_METADATA = []

def log_plot(plot_name, filename, data_info, insights):
    """
    Utility to log plot metadata.
    
    Parameters:
    - plot_name: Tên biểu đồ (human-readable)
    - filename: Tên file đã lưu
    - data_info: Dict chứa thông tin về data (shape, columns, etc.)
    - insights: Insight chính từ biểu đồ
    """
    metadata = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "plot_name": plot_name,
        "filename": filename,
        "data_info": data_info,
        "insights": insights
    }
    PLOT_METADATA.append(metadata)
    
    print("\n" + "="*70)
    print(f"📊 Biểu đồ: {plot_name}")
    print(f"📁 File: {filename}")
    print(f"📈 Dữ liệu: {data_info}")
    print(f"🔍 Insight: {insights}")
    print("="*70 + "\n")

def save_plot_metadata():
    """Lưu tất cả metadata plots vào JSON."""
    metadata_path = DATA_ROOT / "eda_plots_metadata_v1.json"
    with open(metadata_path, "w") as f:
        json.dump(PLOT_METADATA, f, indent=2, ensure_ascii=False)
    print(f"💾 Plot metadata saved: {metadata_path}")

# --- BEACON MAPPING (25 Beacons) ---
MAC_LIST = [
    'F7:7F:78:76:7E:F3', 'C6:CD:5E:3D:2F:BB', 'D6:F4:3A:79:74:63', 'C9:17:55:E2:3E:0E', 'CA:60:AB:EE:EC:7F',
    'D6:51:7F:AB:0E:29', 'CC:54:33:F6:A7:90', 'EB:20:56:87:04:5A', 'EE:E7:46:DC:19:6F', 'C8:5B:BF:37:07:A0',
    'D7:26:F6:A3:44:D2', 'DD:83:B0:27:FD:36', 'E5:CD:4A:36:87:06', 'DC:22:B8:17:4E:B5', 'EA:09:20:80:D6:44',
    'E6:99:D1:EC:C6:81', 'F6:DA:97:C7:D5:28', 'EA:66:A1:12:2C:F4', 'C9:EA:57:8B:0F:80', 'D6:7C:1D:2C:2A:0A',
    'DA:E1:70:5F:44:97', 'DD:10:10:F6:4F:27', 'E6:F3:93:A8:9E:22', 'E6:60:05:1F:88:F9', 'D4:33:FD:F4:C2:A8'
]
BEACON_IDS = [str(i) for i in range(1, 26)]
MAC_TO_BEACON = dict(zip(MAC_LIST, BEACON_IDS))

# --- FINAL OUTPUT ---
print("\n" + "="*70)
print(f"🌍 Environment: {'COLAB' if IS_COLAB else 'LOCAL'}")
print(f"📂 Data Path: {DATA_PATH}")
print(f"📂 Output Path: {OUTPUT_PATH}")
print(f"📡 Beacons: {len(BEACON_IDS)} beacons mapped")
print(f"📊 Figures Directory: {FIGURES_DIR}")
print("="*70)
print("✅ Setup Complete! Ready for EDA.")

In [ ]:
# =============================================================================
# CELL A.2: LOAD FULL DATA (PAIRWISE CONCATENATION)
# =============================================================================
import gc

print("🔄 Loading FULL Raw Data (Pairwise Concatenation)...")
print("="*70)

# --- LOAD LABELS ---
labels_path = Path(DATA_PATH) / "5f_label_loc_train.csv"
df_label = pd.read_csv(labels_path)
print(f"✅ Loaded Labels: {labels_path.name}")
print(f"   Shape: {df_label.shape}")

# --- LOAD BLE FILES (BATCHED) ---
ble_dir = Path(DATA_PATH) / "BLE Data"
ble_files = list(ble_dir.glob("*.csv"))
print(f"\n📡 Total BLE files: {len(ble_files)}")

BATCH_SIZE = 500
print(f"\n🔄 Loading in batches of {BATCH_SIZE}...")

all_batches = []
total_records = 0

for batch_num in range(0, len(ble_files), BATCH_SIZE):
    batch_files = ble_files[batch_num:batch_num + BATCH_SIZE]
    batch_chunks = []
    
    for file_path in batch_files:
        try:
            df_chunk = pd.read_csv(file_path)
            batch_chunks.append(df_chunk)
        except:
            continue
    
    if batch_chunks:
        batch_df = pd.concat(batch_chunks, ignore_index=True)
        all_batches.append(batch_df)
        total_records += len(batch_df)
        
        print(f"   Batch {batch_num//BATCH_SIZE + 1}/{(len(ble_files)-1)//BATCH_SIZE + 1}: "
              f"{len(batch_chunks)} files → {len(batch_df):,} records (Total: {total_records:,})")
        
        del batch_chunks, batch_df
        gc.collect()

# --- PAIRWISE CONCATENATION (KEY FIX!) ---
print(f"\n🔗 Pairwise concatenation ({len(all_batches)} batches)...")

while len(all_batches) > 1:
    new_batches = []
    
    for i in range(0, len(all_batches), 2):
        if i + 1 < len(all_batches):
            # Merge pair
            merged = pd.concat([all_batches[i], all_batches[i+1]], 
                             ignore_index=True, copy=False)
            new_batches.append(merged)
            del all_batches[i], all_batches[i+1]
        else:
            # Odd one out
            new_batches.append(all_batches[i])
    
    all_batches = new_batches
    gc.collect()
    print(f"   → Merged to {len(all_batches)} chunk(s)")

df_ble = all_batches[0]
del all_batches
gc.collect()

print(f"\n✅ All BLE data loaded!")
print(f"   Total records: {len(df_ble):,}")
print(f"   Shape: {df_ble.shape}")
print(f"   Memory: {df_ble.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# --- CONVERT MAC TO BEACON ID ---
print("\n🔧 Converting MAC addresses...")
df_ble['beacon_id'] = df_ble['mac'].map(MAC_TO_BEACON)

unmapped = df_ble['beacon_id'].isna().sum()
if unmapped > 0:
    print(f"⚠️  Dropping {unmapped:,} unmapped ({100*unmapped/len(df_ble):.2f}%)")
    df_ble = df_ble.dropna(subset=['beacon_id'])
else:
    print("✅ All MACs mapped")

print(f"   Final shape: {df_ble.shape}")

# --- STATS ---
print("\n📊 BLE Summary:")
print(f"   Unique beacons: {df_ble['beacon_id'].nunique()}")
print(f"   RSSI: [{df_ble['rssi'].min():.1f}, {df_ble['rssi'].max():.1f}] dBm (mean={df_ble['rssi'].mean():.2f})")

print("\n📊 Label Summary:")
print(f"   Unique locations: {df_label['room'].nunique() if 'room' in df_label.columns else df_label.columns}")
print(f"   Total labels: {len(df_label):,}")

# --- PREVIEW ---
print("\n📋 BLE Preview:")
print(df_ble.head(3))

print("\n📋 Label Preview:")
print(df_label.head(3))

print("\n" + "="*70)
print("✅ FULL data loaded!")
print(f"💾 df_ble: {df_ble.shape}")
print(f"💾 df_label: {df_label.shape}")
print("="*70)

In [ ]:
# =============================================================================
# CELL A.2: LOAD RAW DATA (MEMORY-EFFICIENT VERSION)
# =============================================================================
import gc  # Garbage collection

print("🔄 Loading Raw Data (Memory-Efficient)...")
print("="*70)

# --- LOAD LABELS ---
labels_path = Path(DATA_PATH) / "5f_label_loc_train.csv"
df_label = pd.read_csv(labels_path)
print(f"✅ Loaded Labels: {labels_path.name}")
print(f"   Shape: {df_label.shape}")
print(f"   Columns: {list(df_label.columns)}")

# --- LOAD BLE FILES (BATCHED) ---
ble_dir = Path(DATA_PATH) / "BLE Data"
ble_files = list(ble_dir.glob("*.csv"))
print(f"\n📡 BLE Data directory: {ble_dir.name}/")
print(f"   Total BLE files: {len(ble_files)}")

# Load in batches to avoid memory overflow
BATCH_SIZE = 500
print(f"\n🔄 Loading BLE files in batches of {BATCH_SIZE}...")

all_ble_data = []
total_records = 0

for batch_num in range(0, len(ble_files), BATCH_SIZE):
    batch_files = ble_files[batch_num:batch_num + BATCH_SIZE]
    batch_chunks = []
    
    for file_path in batch_files:
        try:
            df_chunk = pd.read_csv(file_path)
            batch_chunks.append(df_chunk)
        except Exception as e:
            print(f"⚠️  Skip {file_path.name}: {e}")
            continue
    
    # Concatenate batch
    if batch_chunks:
        batch_df = pd.concat(batch_chunks, ignore_index=True)
        all_ble_data.append(batch_df)
        total_records += len(batch_df)
        
        print(f"   Batch {batch_num//BATCH_SIZE + 1}/{(len(ble_files)-1)//BATCH_SIZE + 1}: "
              f"{len(batch_chunks)} files → {len(batch_df):,} records "
              f"(Total: {total_records:,})")
        
        # Free memory
        del batch_chunks, batch_df
        gc.collect()

# Final concatenation
print("\n🔗 Concatenating all batches...")
df_ble = pd.concat(all_ble_data, ignore_index=True)
del all_ble_data
gc.collect()

print(f"✅ All BLE data loaded!")
print(f"   Total records: {len(df_ble):,}")
print(f"   Shape: {df_ble.shape}")
print(f"   Columns: {list(df_ble.columns)}")
print(f"   Memory usage: {df_ble.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# --- CONVERT MAC TO BEACON ID ---
print("\n🔧 Converting MAC addresses to Beacon IDs...")
df_ble['beacon_id'] = df_ble['mac'].map(MAC_TO_BEACON)

# Check unmapped
unmapped_count = df_ble['beacon_id'].isna().sum()
if unmapped_count > 0:
    unmapped_pct = 100 * unmapped_count / len(df_ble)
    print(f"⚠️  Unmapped MACs: {unmapped_count:,} ({unmapped_pct:.2f}%)")
    df_ble = df_ble.dropna(subset=['beacon_id'])
    print(f"   Shape after dropping: {df_ble.shape}")
else:
    print("✅ All MACs mapped successfully")

# --- BASIC STATS ---
print("\n📊 BLE Data Summary:")
print(f"   Unique beacons: {df_ble['beacon_id'].nunique()}")
print(f"   RSSI range: [{df_ble['rssi'].min():.1f}, {df_ble['rssi'].max():.1f}]")
print(f"   Mean RSSI: {df_ble['rssi'].mean():.2f} dBm")

print("\n📊 Label Data Summary:")
print(f"   Unique locations: {df_label['location'].nunique()}")
print(f"   Total labels: {len(df_label):,}")

# --- PREVIEW ---
print("\n📋 BLE Preview:")
print(df_ble[['timestamp', 'mac', 'rssi', 'beacon_id']].head(3))

print("\n📋 Label Preview:")
print(df_label.head(3))

print("\n" + "="*70)
print("✅ Data loaded successfully!")
print(f"💾 df_ble: {df_ble.shape}")
print(f"💾 df_label: {df_label.shape}")
print("="*70)